In [13]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = '4467331.1'

file_list = (#{'date': '10_12_2020', 'scene':'TwoTower_foraging','session':2,'scan':1},
             #{'date': '13_12_2020', 'scene':'TwoTower_foraging','session':2,'scan':2},
             #{'date': '15_12_2020', 'scene':'TwoTower_foraging','session':2,'scan':2},
             {'date': '16_12_2020', 'scene':'TwoTower_foraging','session':2,'scan':4},
             {'date': '17_12_2020', 'scene':'TwoTower_foraging','session':2,'scan':2},
             {'date': '18_12_2020', 'scene':'TwoTower_foraging','session':1,'scan':7},
             {'date': '19_12_2020', 'scene':'TwoTower_foraging','session':2,'scan':2},
             )


In [15]:
def check_and_make_h5s(file_list,runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus5','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus3')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [16]:
for f in file_list:
    
        
    _h5list,_frs = check_and_make_h5s([f,], runh5 = True)
    
    ops_dir = os.path.join(_h5list[0],'suite2p','plane0','ops.npy')
    s2p_dir = os.path.join(_h5list[0],'suite2p','plane0')
    if os.path.exists(ops_dir):
        ops = np.load(os.path.join(s2p_dir,"ops.npy"),allow_pickle=True).all()
        ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
        ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
        np.save(ops_dir,ops)
        
        
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'threshold_scaling':1.,
                                    'h5py_key':'data',
                                    'nchannels':2})
    ops=s2p.run_s2p(ops=ops)


{'date': '16_12_2020', 'scene': 'TwoTower_foraging', 'session': 2, 'scan': 4}
['/mnt/BigDisk/2P_scratch/4467331.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_002_004']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467331.1/16_12_2020/TwoTower_foraging/TwoTower_foraging_002_004/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie [2430,502,776], 223.20 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 2.02, threshold 20.25 


KeyboardInterrupt: 